# Bambu: Google Colab Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/omixlab/bambu-v2/blob/main/notebooks/Bambu%20Google%20Colab%20Tutorial.ipynb)

## Installing RDKit

In [1]:
# run this cell only if you are using this notebook in Google Colab
!pip install kora -q
import kora.install.rdkit

## Installing Bambu

In [3]:
!pip install bambu-qsar==0.0.8

ERROR: Could not find a version that satisfies the requirement bambu-qsar==0.0.8 (from versions: 0.0.5, 0.0.6)
ERROR: No matching distribution found for bambu-qsar==0.0.8


## Downloading data from a PubChem BioAssay

In [ ]:
!bambu-download \
  --pubchem-assay-id 29 \
  --output 29_raw.csv

| __ )  __ _ _ __ ___ | |__  _   _ 
|  _ \ / _` | '_ ` _ \| '_ \| | | |
| |_) | (_| | | | | | | |_) | |_| |
|____/ \__,_|_| |_| |_|_.__/ \__,_|

 50% 13/26 [00:01<00:01,  7.37it/s]
100% 26/26 [00:03<00:00,  7.35it/s]
  6% 27/455 [00:03<00:59,  7.17it/s]
  1% 6/455 [00:00<01:09,  6.48it/s]
 55% 250/455 [00:37<00:31,  6.59it/s]
  2% 8/455 [00:01<01:15,  5.94it/s]
 10% 45/455 [00:06<00:55,  7.35it/s]


## Computing descriptors for the downloaded molecules

In [ ]:
!bambu-preprocess --input 29_raw.csv \
    --train-test-split-percent 0.75 \
    --feature-type descriptors \
    --undersample \
    --output 29_preprocessed.csv \
    --output-preprocessor 29_descriptor_preprocessor.pickle

## Training a predictive model 

In [ ]:
!bambu-train \
	--input-train 29_preprocess_train.csv \
	--input-test 29_preprocess_test.csv \
	--output 29_model.pickle \
	--model-history \
	--max-iter 10 \
	--time-budget 10 \
	--estimators rf extra_tree

## Using the model to analyze new molecules

In [ ]:
!wget -O pubchem_sample.sdf.gz \
  https://github.com/omixlab/bambu-v2/raw/main/tests/pubchem_sample.sdf.gz

!gzip -d -f pubchem_sample.sdf.gz

!bambu-predict \
        --input pubchem_sample.sdf \
        --preprocessor 29_preprocessor.pickle \
        --model 29_model.pickle \
        --output 29_predictions.csv